In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv("../datasets/sms-spam-collection-dataset/spam.csv",
                   encoding="latin-1")
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
data = data.drop(columns=["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"])
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
data = data.rename(index=str, columns={"v1": "labels", "v2": "test"})
data.head()

,labels,test
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [6]:
train.reset_index(drop=True), test.reset_index(drop=True)

(     labels                                               test
 0       ham  No I'm in the same boat. Still here at my moms...
 1      spam  (Bank of Granite issues Strong-Buy) EXPLOSIVE ...
 2       ham     They r giving a second chance to rahul dengra.
 3       ham     O i played smash bros  &lt;#&gt;  religiously.
 4      spam  PRIVATE! Your 2003 Account Statement for 07973...
 ...     ...                                                ...
 4452    ham  I came hostel. I m going to sleep. Plz call me...
 4453    ham                             Sorry, I'll call later
 4454    ham      Prabha..i'm soryda..realy..frm heart i'm sory
 4455    ham                         Nt joking seriously i told
 4456    ham                In work now. Going have in few min.
 
 [4457 rows x 2 columns],
      labels                                               test
 0       ham  Funny fact Nobody teaches volcanoes 2 erupt, t...
 1       ham  I sent my scores to sophas and i had to do sec...
 2      spam

In [7]:
train.shape, test.shape

((4457, 2), (1115, 2))

In [8]:
train.to_csv("../datasets/sms-spam-collection-dataset/train.csv", index=False)
test.to_csv("../datasets/sms-spam-collection-dataset/test.csv", index=False)

In [9]:
!dir ..\datasets\sms-spam-collection-dataset\

 Volume in drive C is OS
 Volume Serial Number is 5453-6B01

 Directory of C:\Users\Ryan\Downloads\test_nlp\datasets\sms-spam-collection-dataset

09/06/2021  10:16 AM    <DIR>          .
09/06/2021  10:16 AM    <DIR>          ..
09/20/2019  02:54 AM           503,663 spam.csv
09/06/2021  02:04 PM            98,560 test.csv
09/06/2021  02:04 PM           386,286 train.csv
               3 File(s)        988,509 bytes
               2 Dir(s)  85,854,773,248 bytes free


In [10]:
import numpy as np
import torch
import torchtext
from torchtext.legacy.data import Field, BucketIterator, TabularDataset

In [11]:
import nltk

nltk.download("punkt")

from nltk import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ryan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
TEXT = torchtext.legacy.data.Field(tokenize=word_tokenize)
LABEL = torchtext.legacy.data.LabelField(dtype=torch.float)

In [13]:
datafields = [("labels", LABEL), ("text", TEXT)]

In [14]:
trn, tst = torchtext.legacy.data.TabularDataset.splits(
    path="../datasets/sms-spam-collection-dataset/",
    train="train.csv",
    test="test.csv",
    format="csv",
    skip_header=True,
    fields=datafields)

In [15]:
trn[:5]

In [16]:
trn[5].__dict__

{'labels': 'ham',
 'text': ['G',
  'says',
  'you',
  'never',
  'answer',
  'your',
  'texts',
  ',',
  'confirm/deny']}

In [17]:
trn[5].text

['G', 'says', 'you', 'never', 'answer', 'your', 'texts', ',', 'confirm/deny']

In [18]:
TEXT.build_vocab(trn, max_size=10500)
LABEL.build_vocab(trn)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 10207
Unique tokens in LABEL vocabulary: 2


In [19]:
print(TEXT.vocab.freqs.most_common(50))

[('.', 3862), ('to', 1750), ('I', 1574), (',', 1468), ('you', 1462), ('?', 1256), ('!', 1134), ('a', 1068), ('the', 946), ('...', 923), ('&', 772), ('i', 760), ('and', 673), ('in', 663), ('is', 647), (';', 641), ('u', 636), ('me', 600), (':', 570), ('..', 544), ('for', 527), ('my', 494), ('of', 471), ('it', 470), ('your', 461), ('have', 395), ('on', 394), (')', 393), ('2', 390), ('that', 385), ("'s", 384), ('now', 321), ("'m", 320), ('are', 316), ('do', 312), ('call', 307), ('at', 301), ('U', 300), ('or', 298), ('not', 295), ("n't", 281), ('be', 275), ('*', 270), ('lt', 267), ('gt', 267), ('with', 267), ('get', 265), ('will', 264), ('so', 257), ('#', 245)]


In [20]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', '.', 'to', 'I', ',', 'you', '?', '!', 'a']


In [21]:
print(LABEL.vocab.stoi)

defaultdict(None, {'ham': 0, 'spam': 1})


In [22]:
batch_size = 64

train_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits(
    (trn, tst),
    batch_size=batch_size,
    sort_key=lambda x: len(x.text),
    sort_within_batch=False,
)

In [23]:
import torch.nn as nn

In [63]:
from IPython.core.debugger import set_trace

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        #set_trace()
        embedded = self.embedding(text)
        output, (hidden, _) = self.rnn(embedded)
        hidden_1D = hidden.squeeze(0)
        assert torch.equal(output[-1, :, :], hidden_1D)
        return self.fc(hidden_1D)

In [67]:
input_dim = len(TEXT.vocab)

embedding_dim = 100

hidden_dim = 256

output_dim = 1

In [68]:
input_dim

10207

In [69]:
model = RNN(input_dim, embedding_dim, hidden_dim, output_dim)

In [70]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=1e-6)

In [71]:
criterion = nn.BCEWithLogitsLoss()

In [72]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        #set_trace()
        predictions = model(batch.text).squeeze(1)

        loss = criterion(predictions, batch.labels)

        rounded_preds = torch.round(torch.sigmoid(predictions))
        correct = (rounded_preds == batch.labels).float()

        acc = correct.sum() / len(correct)

        #set_trace()
        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [76]:
num_epochs = 5

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    print(f"| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% |")

| Epoch: 01 | Train Loss: 0.644 | Train Acc: 85.79% |
| Epoch: 02 | Train Loss: 0.633 | Train Acc: 85.87% |
| Epoch: 03 | Train Loss: 0.622 | Train Acc: 85.90% |
| Epoch: 04 | Train Loss: 0.611 | Train Acc: 85.86% |
| Epoch: 05 | Train Loss: 0.600 | Train Acc: 85.98% |


In [74]:
model.eval()

RNN(
  (embedding): Embedding(10207, 100)
  (rnn): LSTM(100, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [75]:
epoch_loss = 0
epoch_acc = 0

with torch.no_grad():

    for batch in test_iterator:

        predictions = model(batch.text).squeeze(1)

        loss = criterion(predictions, batch.labels)

        rounded_preds = torch.round(torch.sigmoid(predictions))

        correct = (rounded_preds == batch.labels).float()
        acc = correct.sum() / len(correct)

        epoch_loss += loss.item()
        epoch_acc += acc.item()

test_loss = epoch_loss / len(test_iterator)
test_acc = epoch_acc / len(test_iterator)

print(f"| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |")

| Test Loss: 0.690 | Test Acc: 56.86% |
